For CLI testing: 
```python
python compare_df "dev/birthday_loeb/mike_normal.csv" "dev/birthday_loeb/iloy_normal.csv" -l_1 "engine"="python" -l_1 "sep"=";" -l_2 "encoding"="UTF-8" -l_2 "sep"=";" -i "KundenID"
```

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../compare_df"))  # module imports
sys.path.append(os.path.abspath("../compare_df/"))  # intra-module imports

In [2]:
sys.executable

'C:\\Users\\r2d4\\miniconda3\\envs\\py3\\python.exe'

In [3]:
from typing import Dict, Optional
import foos

def main(
    path_1: str,
    path_2: str,
    load_params_1: Optional[Dict[str, str]],
    load_params_2: Optional[Dict[str, str]],
    index_col: Optional[str],
):
    df_1, df_2 = foos.load_csv(
        path_1, path_2, load_params_1, load_params_2, index_col
    )
    df_1, df_2 = foos.impute_missing_values(df_1, df_2)

    if foos.check_if_dataframes_are_equal(df_1, df_2):
        print("Successfully compared, DFs are identical.")
    else:
        if foos.check_for_same_width:
            if not foos.check_for_identical_column_names(df_1, df_2):
                user_input = foos.get_user_input("columns")
                if user_input == "y":
                    df_1, df_2 = foos.handle_different_values(
                        "columns", df_1, df_2
                    )
                else:
                    df_1, df_2 = foos.enforce_column_identity(df_1, df_2)
        else:
            df_1, df_2 = foos.handle_different_values("columns", df_1, df_2)

        df_1, df_2 = foos.sort_columns(df_1, df_2)

        if not foos.check_for_identical_index_values(df_1, df_2):
            df_1, df_2 = foos.handle_different_values("index", df_1, df_2)

        if not foos.check_for_identical_dtypes(df_1, df_2):
            df_1, df_2 = foos.enforce_dtype_identity(df_1, df_2)

        df_diff = foos.compare(df_1, df_2)
        


        if df_diff is not None:
            user_input = foos.get_user_input("output")
            if user_input == "y":
                foos.save_differences_to_xlsx(path_1, df_diff)
        
        # Added for dev purposes only
        return df_1, df_2

## Real-word test-case with different encoding

In [4]:
path_1 = "birthday_loeb/iloy_normal.csv"
path_2 = "birthday_loeb/mike_normal.csv"
params_1 = {'sep': ';', 'encoding': 'UTF-8'}
params_2 = {"sep": ";", "engine": "python"}

In [5]:
# Test with removing non-matching columns
df_a, df_b = main(path_1, path_2, params_1, params_2, "KundenID")

- DF loaded, with original shape of (619, 20)
- DF loaded, with original shape of (618, 20)

The dataframes have the same number of columns, but their names differ. If you want to drop the non-overlapping columns for the comparison, please press 'y'. If you think the data structure of the dataframesis identical and want to enforce the column names to be identical for a 'full' comparison, please press 'n'.y

Found differences in the columns of the two dataframes.
- DF 1 has 2 value(s) in columns that could not be found in the other DF and will be removed:
  - Card Type
  - Jamo_Faktura_Mahnstufe
- DF 2 has 2 value(s) in columns that could not be found in the other DF and will be removed:
  - Jamofak_Mahnstufe
  - CardType

Found differences in the index of the two dataframes.
- DF 1 has 2 value(s) in index that could not be found in the other DF and will be removed:
  - 1272748
  - 1233319
- DF 2 has 1 value(s) in index that could not be found in the other DF and will be removed:
  - 12

In [6]:
df_diff = df_a.ne(df_b)

In [8]:
df_diff.index[df_diff["Street"] > 0]

Int64Index([1300092], dtype='int64', name='KundenID')